In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
nyct=pd.read_csv(r'C:\Users\krist\nyct.csv')
print(nyct)

                            Name               Address      City State
0          Al Hirschfeld Theatre  302 West 45th Street  New York    NY
1             Ambassador Theatre  219 West 49th Street  New York    NY
2      American Airlines Theatre  227 West 42nd Street  New York    NY
3          August Wilson Theatre  245 West 52nd Street  New York    NY
4                Belasco Theatre  111 West 44th Street  New York    NY
5      Bernard B. Jacobs Theatre  242 West 45th Street  New York    NY
6                  Booth Theatre  222 West 45th Street  New York    NY
7               Broadway Theatre         1681 Broadway  New York    NY
8        Brooks Atkinson Theatre  256 West 47th Street  New York    NY
9   Circle in the Square Theatre         1633 Broadway  New York    NY
10                  Cort Theatre  138 West 48th Street  New York    NY
11       Ethel Barrymore Theatre  243 West 47th Street  New York    NY
12        Eugene O'Neill Theatre  230 West 49th Street  New York    NY
13    

In [4]:
nyct['LongAddress'] = nyct[['Address', 'City', 'State']].agg(' '.join, axis=1)
print(nyct)

                            Name               Address      City State  \
0          Al Hirschfeld Theatre  302 West 45th Street  New York    NY   
1             Ambassador Theatre  219 West 49th Street  New York    NY   
2      American Airlines Theatre  227 West 42nd Street  New York    NY   
3          August Wilson Theatre  245 West 52nd Street  New York    NY   
4                Belasco Theatre  111 West 44th Street  New York    NY   
5      Bernard B. Jacobs Theatre  242 West 45th Street  New York    NY   
6                  Booth Theatre  222 West 45th Street  New York    NY   
7               Broadway Theatre         1681 Broadway  New York    NY   
8        Brooks Atkinson Theatre  256 West 47th Street  New York    NY   
9   Circle in the Square Theatre         1633 Broadway  New York    NY   
10                  Cort Theatre  138 West 48th Street  New York    NY   
11       Ethel Barrymore Theatre  243 West 47th Street  New York    NY   
12        Eugene O'Neill Theatre  230 

In [5]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# 2- - create location column
nyct['Geo'] = nyct['LongAddress'].apply(geocode)

# 3 - create longitude, laatitude and altitude from location column (returns tuple)
nyct['LatLong'] = nyct['Geo'].apply(lambda loc: tuple(loc.point) if loc else None)

# 4 - split point column into latitude, longitude and altitude columns
nyct[['latitude', 'longitude', 'altitude']] = pd.DataFrame(nyct['LatLong'].tolist(), index=nyct.index)

In [6]:
nyct2=nyct.drop(columns=['Address', 'City', 'State', 'LongAddress', 'Geo', 'LatLong','altitude'])
print(nyct2)

                            Name   latitude  longitude
0          Al Hirschfeld Theatre  40.759253 -73.989211
1             Ambassador Theatre  40.761236 -73.984990
2      American Airlines Theatre  40.757156 -73.988119
3          August Wilson Theatre  40.763373 -73.984193
4                Belasco Theatre  40.756644 -73.983801
5      Bernard B. Jacobs Theatre  40.758608 -73.987741
6                  Booth Theatre  40.758373 -73.987090
7               Broadway Theatre  40.763398 -73.983355
8        Brooks Atkinson Theatre  40.759975 -73.986966
9   Circle in the Square Theatre  40.762093 -73.984656
10                  Cort Theatre  40.759103 -73.983090
11       Ethel Barrymore Theatre  40.760122 -73.986135
12        Eugene O'Neill Theatre  40.761105 -73.985740
13       George Gershwin Theatre  40.762362 -73.985204
14     Gerald Schoenfeld Theatre  40.758486 -73.987438
15                 Hayes Theater  40.757851 -73.987842
16                Hudson Theatre  40.757083 -73.984792
17        

In [7]:
nyctmap = folium.Map(location=[40.7590, -73.9845], tiles="OpenStreetMap", zoom_start=15)

nyct2.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                                              radius=10, popup=row['Name'])
                                             .add_to(nyctmap), axis=1)

nyctmap